## Data Partitioning in PySpark In-depth

Data partitioning is critical to data processing performance especially for large volume of data processing in Spark.  
Partitions in Spark won’t span across nodes though one node can contains more than one partitions.  
When processing, Spark assigns one task for each partition and each worker threads can only process one task at a time.  
Thus, 
* with too few partitions, the application won’t utilize all the cores available in the cluster and it can cause data skewing problem; 
* with too many partitions, it will bring overhead for Spark to manage too many small tasks.

In [ ]:
dbutils.library.restartPython() # Removes Python state, but some libraries might not work without calling this command.dbutils.restartPython()
print(spark.version)

3.0.1

#### Load libraries

In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField, ArrayType, MapType, DoubleType
from pyspark.sql.functions import lit, col, expr, when, sum, avg, max, min, mean, count, udf, explode, concat_ws, year, month, dayofmonth

#### Create Spark session

In [ ]:
spark = SparkSession.builder.appName('PySpark Partitioning In-depth').getOrCreate()

#### Create Dataframe

In [ ]:
from datetime import date, timedelta

start_date = date(2019, 1, 1)
data = []

for i in range(0, 50):
  data.append({'Country': 'CN', 'Date': start_date + timedelta(days=i), 'Amount': 10+i})
  data.append({'Country': 'AU', 'Date': start_date + timedelta(days=i), 'Amount': 10+i})

schema = StructType([
  StructField('Country', StringType(), nullable=False),
  StructField('Date', DateType(), nullable=False),
  StructField('Amount', IntegerType(), nullable=False)]
)

df = spark.createDataFrame(data=data, schema=schema)
df.printSchema()
df.show()

root
-- Country: string (nullable = false)
-- Date: date (nullable = false)
-- Amount: integer (nullable = false)

+-------+----------+------+
Country| Date|Amount|
+-------+----------+------+
 CN|2019-01-01| 10|
 AU|2019-01-01| 10|
 CN|2019-01-02| 11|
 AU|2019-01-02| 11|
 CN|2019-01-03| 12|
 AU|2019-01-03| 12|
 CN|2019-01-04| 13|
 AU|2019-01-04| 13|
 CN|2019-01-05| 14|
 AU|2019-01-05| 14|
 CN|2019-01-06| 15|
 AU|2019-01-06| 15|
 CN|2019-01-07| 16|
 AU|2019-01-07| 16|
 CN|2019-01-08| 17|
 AU|2019-01-08| 17|
 CN|2019-01-09| 18|
 AU|2019-01-09| 18|
 CN|2019-01-10| 19|
 AU|2019-01-10| 19|
+-------+----------+------+
only showing top 20 rows

8

In [ ]:
table_name = 'temp.partitions_testing'
table_path = f'/mnt/{table_name}'

#### Write data frame to file system

In [ ]:
(df
.write
.format('delta')
.mode('overwrite')
.save(table_path))

#### Check Number of Partitions

In [ ]:
print(df.rdd.getNumPartitions())

8

In [ ]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing

total 24K
drwxrwxrwx 2 root root 4.0K Aug 5 12:04 .
drwxrwxrwx 2 root root 4.0K Aug 5 12:04 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 _delta_log
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00000-2d694c51-f2df-4c93-b8cf-bb027bea21ee-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00001-43040274-e3af-4710-9bb2-b9dc3fccfb86-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00002-e23159d2-bc4e-4a29-a1b6-9a3fc66ff9bd-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00003-3b7ec128-f87c-4322-98fe-a2a7c9596065-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00004-390968e4-7bae-4ef6-b4fd-b02dcf31a4b2-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00005-30317160-32af-4f73-b422-169f23f3ea5f-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00006-894a9499-72d3-441a-b07b-8b6405e95e70-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00007-70891a56-310e-41d8-a5e1-0c79636f2956-c000.snappy.parquet

#### Repartitioning with coalesce function

Similar to coalesce defined on an RDD, this operation results in a narrow dependency, e.g. if you go from 1000 partitions to 100 partitions, there will not be a shuffle, instead each of the 100 new partitions will claim 10 of the current partitions. If a larger number of partitions is requested, it will stay at the current number of partitions.

However, if you’re doing a drastic coalesce, e.g. to numPartitions = 1, this may result in your computation taking place on fewer nodes than you like (e.g. one node in the case of numPartitions = 1). To avoid this, you can call repartition(). This will add a shuffle step, but means the current upstream partitions will be executed in parallel (per whatever the current partitioning is).

In [ ]:
df = df.coalesce(16)

(df
.write
.format('delta')
.mode('overwrite')
.save(table_path))

In [ ]:
print(df.rdd.getNumPartitions()) # still 8
# bt ls will return 16

8

In [ ]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing

total 36K
drwxrwxrwx 2 root root 4.0K Aug 5 13:09 .
drwxrwxrwx 2 root root 4.0K Aug 5 12:04 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 _delta_log
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00000-2d694c51-f2df-4c93-b8cf-bb027bea21ee-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00000-a3380617-4f28-4e7d-bbf7-a6292ee574d4-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00001-43040274-e3af-4710-9bb2-b9dc3fccfb86-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00001-4e22d4de-26e7-4ae3-b12b-a6de7c570fd7-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00002-0b86b528-c905-4fbc-ac2c-ff963d5f2967-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00002-e23159d2-bc4e-4a29-a1b6-9a3fc66ff9bd-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00003-3b7ec128-f87c-4322-98fe-a2a7c9596065-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00003-a1555ad9-3ce4-42a0-b045-684da007b691-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00004-390968e4-7bae-4ef6-b4fd-b02dcf31a4b2-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00004-877d577d-9058-4aaf-9197-d34ff00bb4eb-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00005-30317160-32af-4f73-b422-169f23f3ea5f-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00005-7ccd077d-059b-4620-80e7-bef46eb697b8-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00006-6b4451a5-c162-4dbd-93ac-b74db8270300-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00006-894a9499-72d3-441a-b07b-8b6405e95e70-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00007-70891a56-310e-41d8-a5e1-0c79636f2956-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00007-ad96b961-b7b1-47fe-b2c1-24e76a386ec3-c000.snappy.parquet

In [ ]:
df = df.coalesce(4)

(df
.write
.format('delta')
.mode('overwrite')
.save(table_path))

In [ ]:
print(df.rdd.getNumPartitions()) # will return 4
# in delta there will be 20 files

4

In [ ]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing

total 42K
drwxrwxrwx 2 root root 4.0K Aug 5 13:11 .
drwxrwxrwx 2 root root 4.0K Aug 5 12:04 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 _delta_log
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00000-2d694c51-f2df-4c93-b8cf-bb027bea21ee-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00000-a3380617-4f28-4e7d-bbf7-a6292ee574d4-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:11 part-00000-beba0ea9-0c60-4fa4-a983-56190e64b99e-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:10 part-00001-357e7fb0-2d8f-4399-93e9-67a596853170-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00001-43040274-e3af-4710-9bb2-b9dc3fccfb86-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00001-4e22d4de-26e7-4ae3-b12b-a6de7c570fd7-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00002-0b86b528-c905-4fbc-ac2c-ff963d5f2967-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:11 part-00002-0bbd299a-40d1-4c5d-b2c3-27f25f37cf13-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00002-e23159d2-bc4e-4a29-a1b6-9a3fc66ff9bd-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00003-3b7ec128-f87c-4322-98fe-a2a7c9596065-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.2K Aug 5 13:10 part-00003-4e013464-705c-4b72-8391-018de63e0564-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00003-a1555ad9-3ce4-42a0-b045-684da007b691-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00004-390968e4-7bae-4ef6-b4fd-b02dcf31a4b2-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00004-877d577d-9058-4aaf-9197-d34ff00bb4eb-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00005-30317160-32af-4f73-b422-169f23f3ea5f-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00005-7ccd077d-059b-4620-80e7-bef46eb697b8-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00006-6b4451a5-c162-4dbd-93ac-b74db8270300-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00006-894a9499-72d3-441a-b07b-8b6405e95e70-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 12:04 part-00007-70891a56-310e-41d8-a5e1-0c79636f2956-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:09 part-00007-ad96b961-b7b1-47fe-b2c1-24e76a386ec3-c000.snappy.parquet

In [ ]:
%sh
rm -rf /dbfs/mnt/

In [ ]:
# after dr]eleting the delta table
# there should be 4 files
(df
.write
.format('delta')
.mode('overwrite')
.save(table_path))

In [ ]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing

total 18K
drwxrwxrwx 2 root root 4.0K Aug 5 13:14 .
drwxrwxrwx 2 root root 4.0K Aug 5 13:14 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 _delta_log
-rwxrwxrwx 1 root root 1.1K Aug 5 13:14 part-00000-995445d6-10d7-4a0e-b011-4974cc079c8b-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:14 part-00001-66a304f6-9ade-49b5-a607-9affaebfbb32-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.1K Aug 5 13:14 part-00002-38798824-79b3-4de7-b82c-7119cc17facf-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.2K Aug 5 13:14 part-00003-ee449317-86f5-4ee8-8bf7-b9b23a0a99a5-c000.snappy.parquet

In [ ]:
%sh
rm -rf /dbfs/mnt/

#### Repartitioning with repartition function

Returns a new DataFrame partitioned by the given partitioning expressions. The resulting DataFrame is hash partitioned.  

##### Repartition by number

Pypark will try to evenly distribute the data to each partitions.  
If the total partition number is greater than the actual record count (or RDD size), some partitions will be empty.

In [ ]:
df = df.repartition(10)

(df
.write
.format('delta')
.mode('overwrite')
.save(table_path))

In [ ]:
print(df.rdd.getNumPartitions())

10

In [ ]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing

total 22K
drwxrwxrwx 2 root root 4.0K Aug 5 13:16 .
drwxrwxrwx 2 root root 4.0K Aug 5 13:15 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 _delta_log
-rwxrwxrwx 1 root root 1005 Aug 5 13:16 part-00000-c1da2fe5-af5b-4e7f-bb82-86c67a77befc-c000.snappy.parquet
-rwxrwxrwx 1 root root 1006 Aug 5 13:16 part-00001-c97726eb-1c5a-4c96-be4c-332a136a475a-c000.snappy.parquet
-rwxrwxrwx 1 root root 1014 Aug 5 13:16 part-00002-49d74ddc-5e54-485a-8664-66ab3a20e2ba-c000.snappy.parquet
-rwxrwxrwx 1 root root 1006 Aug 5 13:16 part-00003-eef85a7e-db77-4a4e-b1a7-0787b0e2bab5-c000.snappy.parquet
-rwxrwxrwx 1 root root 998 Aug 5 13:15 part-00004-6cf58539-4f96-4cd5-9a5a-a4257b2463b1-c000.snappy.parquet
-rwxrwxrwx 1 root root 998 Aug 5 13:15 part-00005-cf9eb24e-d27e-42c4-804a-3740a9d0d99d-c000.snappy.parquet
-rwxrwxrwx 1 root root 1006 Aug 5 13:15 part-00006-4e712e42-202f-4e8d-831b-74f2dcccef23-c000.snappy.parquet
-rwxrwxrwx 1 root root 999 Aug 5 13:16 part-00007-1aa2d88b-3b6f-4074-940e-02068d098cba-c000.snappy.parquet
-rwxrwxrwx 1 root root 1005 Aug 5 13:15 part-00008-d528a2a3-7e11-415c-8ecd-15214064450e-c000.snappy.parquet
-rwxrwxrwx 1 root root 1007 Aug 5 13:15 part-00009-8155ddb1-d650-43d6-9da0-ad5a363075e3-c000.snappy.parquet

In [ ]:
%sh
rm -rf /dbfs/mnt/

##### Repartition by column

The below scripts will create 200 partitions (Spark by default create 200 partitions).  
However only three sharded files are generated:

* One file stores data for CN country.
* Another file stores data for AU country.
* The other one is empty.

In [ ]:
df = df.repartition('Country')

In [ ]:
(df
.write
.format('delta')
.mode('overwrite')
.save(table_path))

In [ ]:
print(df.rdd.getNumPartitions())

200

In [ ]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing

total 16K
drwxrwxrwx 2 root root 4.0K Aug 5 13:19 .
drwxrwxrwx 2 root root 4.0K Aug 5 13:19 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 _delta_log
-rwxrwxrwx 1 root root 516 Aug 5 13:19 part-00000-df00d60c-50d0-4143-b737-71e975b5ce79-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.4K Aug 5 13:19 part-00028-b3779a05-d944-445b-ae1e-ed020de7f51a-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.4K Aug 5 13:19 part-00064-122fc97f-4234-4fbd-bfc9-a6195e9da163-c000.snappy.parquet

In [ ]:
%sh
rm -rf /dbfs/mnt/

#### Partition by multiple columns

In [ ]:
df = (
  df
  .withColumn('Year', year('Date'))
  .withColumn('Month', month('Date'))
  .withColumn('Day', dayofmonth('Date'))
)
df = df.repartition('Year', 'Month', 'Day', 'Country')

In [ ]:
(df
.write
.format('delta')
.mode('overwrite')
.save(table_path))

In [ ]:
print(df.rdd.getNumPartitions())

200

In [ ]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing

total 138K
drwxrwxrwx 2 root root 4.0K Aug 5 13:24 .
drwxrwxrwx 2 root root 4.0K Aug 5 13:24 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 _delta_log
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00000-4bdb1713-1c45-47c7-8cff-9cfd1f31d0be-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00005-6d2cafda-e11d-4d72-87ba-5f7815aa6869-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00017-65c1cb56-acdd-4ad2-bd5f-615ae93f7af9-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.7K Aug 5 13:24 part-00018-f8ad735d-8966-4e8c-bae1-10c8458d3374-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.7K Aug 5 13:24 part-00021-c07ec0af-6307-4dff-8904-42b9b3fbf0ce-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00024-5fa02b88-ef0e-4cdf-831a-004101d2ef5f-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00025-62c782d4-3538-4aad-b201-a67a8f080d16-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.7K Aug 5 13:24 part-00026-680accbc-f523-414e-9be3-7cc7eebb4e46-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.7K Aug 5 13:24 part-00027-a944d1a7-274d-472b-ac8b-17f2f5bf3140-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00031-b8575506-a708-4045-b87b-fd255a4be757-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00032-3f96c640-059a-4746-8de7-1ef3e851bc36-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.6K Aug 5 13:24 part-00033-d243a24e-4abe-4547-9ea8-18d221ecc8a7-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00036-8cf6adf7-6094-4001-9691-46873b9d9998-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00037-a4bc4858-06d9-4f96-9002-895551481a87-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00038-d692a9ea-12aa-420e-9fb9-a3a6c0272f3b-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00039-55f2443e-e628-491c-bf7b-def656bf116b-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.6K Aug 5 13:24 part-00041-32e9d8d3-83dd-4c57-b0f4-440b6fe2d959-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.7K Aug 5 13:24 part-00046-87d2d8a9-17e5-4673-91c1-64fdf24a1c1e-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00052-ed380bd2-074a-40f5-8ab2-5dc1735dfb63-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00053-0c51a741-d4aa-4815-a617-4a9707a35bcf-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00054-b4e2a59b-d088-4968-8426-b3faca737ad8-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.7K Aug 5 13:24 part-00060-5187892c-d0dd-4620-9e27-ecc9831f1cc6-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00062-f8bcf144-5184-464d-8457-d07fa9d2002c-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00063-1bd77c36-5c7c-4d70-948f-462fa903712f-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00066-194ab90c-e6b3-4de9-b4ec-534432800848-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00067-c3209a52-3530-47d1-8fe3-53ffcd0c0f3f-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00068-88e20a7a-7d87-4a21-845e-d04f1dacf922-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00071-af1c7038-3864-4ff7-9b90-29ddd895c79d-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00072-c6506a82-73f0-4c67-8965-9e5bb77c2741-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00074-5f1308cb-c756-4d1d-943e-d213f7fd6523-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00076-94e70f66-e6c4-41e2-a5bc-b0d19ddf56f6-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00078-0b60b675-c054-4c6f-b372-67c3fbd00961-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00081-0d6f8df9-80f4-46c4-acad-102ca088e93c-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00082-15d58e60-cae7-4c06-b96e-dbc62a171ff5-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00084-7864239f-0d79-48f7-8eaf-8da408fda05c-c000.snappy.parquet
-rwxrwxrwx 1 root root 1.5K Aug 5 13:24 part-00085-48822a01-0671-437e-8f

In [ ]:
%sh
rm -rf /dbfs/mnt/

#### Match repartition keys with write partition keys

In [ ]:
(df
.write
.partitionBy('Year', 'Month', 'Day', 'Country')
.format('delta')
.mode('overwrite')
.save(table_path))

In [ ]:
%sh
find /dbfs/mnt/temp.partitions_testing -maxdepth 4 -type d

/dbfs/mnt/temp.partitions_testing
/dbfs/mnt/temp.partitions_testing/Year=2019
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=1
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=1/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=1/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=10
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=10/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=10/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=11
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=11/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=11/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=12
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=12/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=12/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=13
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=13/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=13/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=14
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=14/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=14/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=15
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=15/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=15/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=16
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=16/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=16/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=17
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=17/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=17/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=18
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=18/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=18/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=19
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=19/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=19/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=2
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=2/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=2/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=20
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=20/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=20/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=21
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=21/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=21/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=22
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=22/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=22/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=23
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=23/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=23/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=24
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=24/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=24/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=25
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=25/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=25/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=26
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=26/Country=AU
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=26/Country=CN
/dbfs/mnt/temp.partitions_testing/Year=2019/Month=1/Day=27
/dbfs/mnt/temp.partitions_testing/Ye

#### Read from partitioned data

In [ ]:
df = spark.read.format('delta').load(f'{table_path}/Year=2019/Month=2/Day=1/Country=CN')
print(df.rdd.getNumPartitions())
df.show()

1
+-------+----------+------+----+-----+---+
Country| Date|Amount|Year|Month|Day|
+-------+----------+------+----+-----+---+
 CN|2019-02-01| 41|2019| 2| 1|
+-------+----------+------+----+-----+---+

In [ ]:
df = spark.read.format('delta').load(f'{table_path}/Year=2019/Month=2')
print(df.rdd.getNumPartitions())
df.show()

20
+-------+----------+------+----+-----+---+
Country| Date|Amount|Year|Month|Day|
+-------+----------+------+----+-----+---+
 AU|2019-02-05| 45|2019| 2| 5|
 CN|2019-02-06| 46|2019| 2| 6|
 AU|2019-02-08| 48|2019| 2| 8|
 CN|2019-02-18| 58|2019| 2| 18|
 AU|2019-02-13| 53|2019| 2| 13|
 AU|2019-02-03| 43|2019| 2| 3|
 AU|2019-02-19| 59|2019| 2| 19|
 AU|2019-02-07| 47|2019| 2| 7|
 CN|2019-02-19| 59|2019| 2| 19|
 AU|2019-02-01| 41|2019| 2| 1|
 CN|2019-02-11| 51|2019| 2| 11|
 AU|2019-02-16| 56|2019| 2| 16|
 CN|2019-02-16| 56|2019| 2| 16|
 CN|2019-02-01| 41|2019| 2| 1|
 AU|2019-02-06| 46|2019| 2| 6|
 CN|2019-02-17| 57|2019| 2| 17|
 AU|2019-02-09| 49|2019| 2| 9|
 AU|2019-02-02| 42|2019| 2| 2|
 CN|2019-02-14| 54|2019| 2| 14|
 AU|2019-02-11| 51|2019| 2| 11|
+-------+----------+------+----+-----+---+
only showing top 20 rows

In [ ]:
%sh
rm -rf /dbfs/mnt/

#### The end of the notebook